In [1]:
import os
from tqdm import tqdm
from huggingface_hub import login
import torch
import torch.nn as nn
import math
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px
from jaxtyping import Float
from functools import partial
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, FactoredMatrix

ImportError: cannot import name 'load_dataset' from partially initialized module 'datasets' (most likely due to a circular import) (/Users/eric3/LocalDocuments/CS282r/saes2822r/datasets.py)

In [ ]:
with open("/Users/cole/cs2822r/saes2822r/access.tok", "r") as file:
    access_token = file.read()
    login(token=access_token)

if torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Device: {device}")

In [ ]:
from datasets import load_dataset  
import transformer_lens
from transformer_lens import HookedTransformer
from sae_lens import SAE, HookedSAETransformer

torch.set_grad_enabled(False)

In [ ]:
# load gpt-2-small (for testing)
model = HookedSAETransformer.from_pretrained("gpt2-small", device=device)

# load sae on res stream of gpt-2-small, plus cfg and sparsity val (for testing)
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gpt2-small-res-jb",
    sae_id = "blocks.7.hook_resid_pre",
    device = device
)

In [ ]:
sae.use_error_term

top_neurons_neg = {}
top_neurons_pos = {}

for example in negative_set:
    _, cache = model.run_with_cache_with_saes(example, saes=[sae])

    # get top 15 firing sae neurons
    vals, inds = torch.topk(cache['blocks.7.hook_resid_pre.hook_sae_acts_post'][0, -1, :], 15)

    for datapoint in zip(inds, vals):
        if datapoint[0] in top_neurons_neg:
            top_neurons_neg[datapoint[0]].append(datapoint[1])
        else:
            top_neurons_neg[datapoint[0]] = [datapoint[1]]
    

for example in positive_set:
    _, cache = model.run_with_cache_with_saes(example, saes=[sae])

    # get top 15 firing sae neurons
    vals, inds = torch.topk(cache['blocks.7.hook_resid_pre.hook_sae_acts_post'][0, -1, :], 15)

    for datapoint in zip(inds, vals):
        if datapoint[0] in top_neurons_pos:
            top_neurons_pos[datapoint[0]].append(datapoint[1])
        else:
            top_neurons_pos[datapoint[0]] = [datapoint[1]]

In [ ]:
# average activations over each top case, sends to
# top_neurons_neg/pos = {idx: avg_act, idx2:avg_act2, ...}
for entry in top_neurons_neg:
    top_neurons_neg[entry] = math.mean(top_neurons_neg[entry])

for entry in top_neurons_pos:
    top_neurons_pos[entry] = math.mean(top_neurons_pos[entry])

for entry in top_neurons_pos:
    

In [ ]:
# train classifier on top activations